In [245]:
import pandas as pd
import numpy as np
import random

In [246]:
classes = pd.read_csv("classes.csv").to_numpy()
list_of_ingredients = pd.read_csv("ingredients.csv").to_numpy()
pairings = pd.read_csv("pairings.csv").to_numpy()


[1 1 1 50 50 5 5 5 5 10 10 10 10 10 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5
 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5
 50 50 50 50 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10
 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10
 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 5 5 5 5 5 5 5 5 5 5
 5 1 1 1 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10
 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10]


In [247]:
def generate_random_recipe(list_of_ingredients, classes):
    recipe = []
    for cl in classes:
        class_ing = list_of_ingredients[list_of_ingredients[:,1] == cl[0]]
        np.random.shuffle(class_ing)
        
        no_elements = np.random.randint(cl[1],cl[2]+1) 

        ingredients = class_ing[:no_elements]
       
        quantities = []
        for ing in ingredients:
            qt = random.randrange(ing[3],ing[2],ing[4])
            quantities.append(qt)


        quantities = np.reshape(np.array(quantities),(len(ingredients),1))
        ing_quan = np.hstack((ingredients,quantities))
        if(len(recipe) == 0):
            recipe = ing_quan
        else:
            recipe = np.concatenate((recipe,ing_quan))
        
    return np.array(recipe)


In [248]:
def taste_loss(recipe,pairings):
    count = 0
    for i in range(len(recipe)):
        for j in range(i+1,len(recipe)):
            ing1 = recipe[i,0]
            qt1 = (recipe[i,-1] - recipe[i,3]+1)/(recipe[i,2] - recipe[i,3])
            ing2 = recipe[j,0]
            qt2 = (recipe[j,-1] - recipe[j,3]+1)/(recipe[j,2] - recipe[j,3])
            raport = qt1/qt2
            if(qt1 > qt2):
                raport = qt2/qt1 

            for el in pairings:
                if((el[0] == ing1 and el[1] == ing2) or (el[1] == ing1 and el[0] == ing2)):

                    count +=raport
                    break
    return count/len(recipe)


In [271]:
def mutation(recipe, classes, list_of_ingredients, p1 = 0.25,p2 = 0.25, p3 = 0.25):
    new_recipe = []
    for cl in classes:
        cl_recipe = recipe[recipe[:,1] == cl[0]]
        
        cl_all_ing = list_of_ingredients[list_of_ingredients[:,1]==cl[0]]
        if(len(cl_recipe) > cl[1] and np.random.random()<p1):
            position = np.random.randint(len(cl_recipe))
            cl_recipe=np.delete(cl_recipe,position,axis=0)
        



        for ing in cl_recipe:
            if(np.random.random()<p2):
                ing[-1] = random.randrange(ing[3],ing[2],ing[4])
            new_recipe.append(ing)
        
        
        if(len(cl_recipe) < cl[2] and np.random.random()<p3):
            position = np.random.randint(len(cl_all_ing))
            ing = cl_all_ing[position]
            qt = random.randrange(ing[3],ing[2],ing[4])
            add = True
            for i in range(len(new_recipe)):
                if(new_recipe[i][0] == ing[0]):
                    add = False
                    break
            if(add): 
                new_recipe.append(np.hstack((ing,qt)))
            
     
    return np.array(new_recipe)

In [250]:
def crossover(recipe1,recipe2,classes):
    ofs1 = []
    ofs2 = []
    for cl in classes:
        cl_recipe1 = recipe1[recipe1[:,1] == cl[0]]
        cl_recipe2 = recipe2[recipe2[:,1] == cl[0]]
        crossover_mask = np.array([np.random.randint(0,2) for _ in range(cl[2])])
        
        for i in range(cl[2]):
            if(crossover_mask[i] == 0):
                if(i < len(cl_recipe1)):
                    ofs1.append(cl_recipe1[i])
                if(i < len(cl_recipe2)):
                    ofs2.append(cl_recipe2[i])
            if(crossover_mask[i] == 1):
                if(i < len(cl_recipe2)):
                    ofs1.append(cl_recipe2[i])
                if(i < len(cl_recipe1)):
                    ofs2.append(cl_recipe1[i])
       
    ofs1 = np.array(ofs1)
    ofs2 = np.array(ofs2)

    for i in range(len(ofs1)):
        j=i+1
        while(j<len(ofs1)):
            if(ofs1[i][0] == ofs1[j][0]):
                ofs1[i][-1] = int((ofs1[i][-1]+ ofs1[j][-1])/2/ofs1[i][4]) * ofs1[i][4]
                ofs1 = np.delete(ofs1,j,axis=0)
            else:
                j=j+1
    
    for i in range(len(ofs2)):
        j=i+1
        while(j<len(ofs2)):
            if(ofs2[i][0] == ofs2[j][0]):
                ofs2[i][-1] = int((ofs2[i][-1]+ ofs2[j][-1])/2/ofs2[i][4]) * ofs2[i][4]
                ofs2 = np.delete(ofs2,j,axis=0)
            else:
                j=j+1
    return ofs1,ofs2

In [251]:
def get_parents(population, losses):
    aux = losses - np.min(losses)
    probabilities = aux/ np.sum(aux)
    index1 = np.random.choice(len(losses),p=probabilities,size=1,replace=True)
    index2 = np.random.choice(len(losses),p=probabilities,size=1,replace=True)
    while(index2 == index1):
        index2 = np.random.choice(len(losses),p=probabilities,size=1,replace=True)
    return (population[index1],population[index2])

In [252]:

class GA:
    def __init__(self, classes, list_of_ingredients, pairings) -> None:
        self.classes = classes
        self.list_of_ingredients = list_of_ingredients
        self.pairings = pairings
    

    def generate(self, population_size=100, offsprings_size =100, no_epochs = 10, p1 =0.25, p2=0.25, p3=0.25):
        population = []
        losses = []
        for _ in range(population_size):
            recipe = generate_random_recipe(self.list_of_ingredients,self.classes)
            loss = taste_loss(recipe,self.pairings)
            population.append(recipe)
            losses.append(loss)
        
        population = np.array(population,dtype=object)
        losses = np.array(losses)
        for _ in range(no_epochs):
            # print("ceva")
            new_offsprings = []
            new_losses = []
            count = 0
            while count < offsprings_size:
                (parent1,parent2) = get_parents(population,losses)
                
                parent1 = parent1[0]
                parent2 = parent2[0]

                

                ofs1,ofs2 = crossover(parent1,parent2,classes)
                

                ofs1 = mutation(ofs1,classes,list_of_ingredients,p1 =p1, p2=p2, p3=p3)
                ofs2 = mutation(ofs2,classes,list_of_ingredients,p1 =p1, p2=p2, p3=p3)
       
                loss1 = taste_loss(ofs1,self.pairings)
                loss2 = taste_loss(ofs2,self.pairings)


                new_offsprings.append(ofs1)
                new_losses.append(loss1)
                new_offsprings.append(ofs2)
                new_losses.append(loss2)
                count +=2
            new_offsprings = np.array(new_offsprings,dtype=object)
            new_losses = np.array(new_losses)

            
            all_pop = np.concatenate((population,new_offsprings))
            
            all_losses = np.concatenate((losses,new_losses))

            index = np.argsort(-all_losses)
            all_pop = all_pop[index]
            all_losses = all_losses[index]
            
            print("Epoch "+ str(_) + " best recipe loss: " + str(all_losses[np.argmax(all_losses)]))
            population = all_pop[:population_size]
            losses = all_losses[:population_size]
        return population[np.argmax(losses)]
        


In [272]:
cookie_generator = GA(classes,list_of_ingredients,pairings)

no_recipes = 1
for _ in range(no_recipes):
    recipe = cookie_generator.generate(population_size=100, offsprings_size =100, no_epochs = 10, p1 =0.25, p2=0.25, p3=0.2)
    for ing in recipe:
        print(ing[0] + "   " + str(ing[-1]))
    

Epoch 0 best recipe loss: 0.5955971031996111
after mut
Epoch 1 best recipe loss: 0.5955971031996111
Epoch 2 best recipe loss: 0.5955971031996111
Epoch 3 best recipe loss: 0.5977919946190373
Epoch 4 best recipe loss: 0.902219880013717
Epoch 5 best recipe loss: 0.902219880013717
Epoch 6 best recipe loss: 1.002855904380249
Epoch 7 best recipe loss: 1.002855904380249
Epoch 8 best recipe loss: 1.002855904380249
Epoch 9 best recipe loss: 1.002855904380249
cloves   50
cinnamon   75
raspberry   190
banana   170
cashew   165
oil   60
baking powder   3
whole egg   3
maple syrup   80
honey   70
rye flour   250
